# 04 - MLflow: Tracking de Experimentos

**Objetivo:** Trackear experimentos de ML para comparar modelos y versionar el mejor.

**Input:** `data/processed/dataset_modelo.csv`

**Output:** Experimentos en MLflow + modelo registrado

## 1. Setup

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# ML
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# MLflow
import mlflow
import mlflow.sklearn

# Config
np.random.seed(42)
os.chdir("/Users/javiermondragon/Documents/data_projects/inflation-predictor")

print("Setup completo")
print(f"MLflow version: {mlflow.__version__}")

Setup completo
MLflow version: 3.10.0


## 2. Configurar MLflow

In [4]:
# Crear carpeta para MLflow
os.makedirs("mlruns", exist_ok=True)

# Configurar tracking local
mlflow.set_tracking_uri("file:./mlruns")

# Crear o usar experimento existente
experiment_name = "inflation-prediction"
mlflow.set_experiment(experiment_name)

print(f"Experimento: {experiment_name}")
print(f"Tracking URI: {mlflow.get_tracking_uri()}")

/Users/javiermondragon/Documents/data_projects/inflation-predictor/venv/lib/python3.10/site-packages/mlflow/tracking/_tracking_service/utils.py:184: FutureWarning: The filesystem tracking backend (e.g., './mlruns') is deprecated as of February 2026. Consider transitioning to a database backend (e.g., 'sqlite:///mlflow.db') to take advantage of the latest MLflow features. See https://mlflow.org/docs/latest/self-hosting/migrate-from-file-store for migration guidance.
  return FileStore(store_uri, store_uri)
2026/02/20 20:25:50 INFO mlflow.tracking.fluent: Experiment with name 'inflation-prediction' does not exist. Creating a new experiment.


Experimento: inflation-prediction
Tracking URI: file:./mlruns


## 3. Cargar y Preparar Datos

In [5]:
# Cargar datos
df = pd.read_csv('data/processed/dataset_modelo.csv', parse_dates=['date'])
print(f"Dataset: {len(df)} filas")

Dataset: 479 filas


In [6]:
# Feature Engineering (mismo que notebook 03)
predictores = ['fed_rate', 'oil_price', 'gold_price']
lags = [1, 3, 6, 12]
ventanas = [3, 6, 12]

# Target: inflación
df['inflation'] = df['cpi'].pct_change() * 100

# Lags de predictores
for var in predictores:
    for lag in lags:
        df[f'{var}_lag{lag}'] = df[var].shift(lag)

# Lags de inflación
for lag in lags:
    df[f'inflation_lag{lag}'] = df['inflation'].shift(lag)

# Promedios móviles
for var in predictores:
    for ventana in ventanas:
        df[f'{var}_ma{ventana}'] = df[var].rolling(window=ventana).mean()

for ventana in ventanas:
    df[f'inflation_ma{ventana}'] = df['inflation'].shift(1).rolling(window=ventana).mean()

# Variaciones porcentuales
for var in predictores:
    df[f'{var}_pct_1m'] = df[var].pct_change(1) * 100
    df[f'{var}_pct_12m'] = df[var].pct_change(12) * 100

# Features temporales
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['quarter'] = df['date'].dt.quarter

# Limpiar NaN
df_clean = df.dropna().reset_index(drop=True)
print(f"Dataset limpio: {len(df_clean)} filas")

Dataset limpio: 466 filas


In [7]:
# Preparar X e y
target = 'inflation'
excluir = ['date', 'cpi', 'inflation']
features = [col for col in df_clean.columns if col not in excluir]

X = df_clean[features]
y = df_clean[target]

# Split temporal
split_idx = int(len(df_clean) * 0.8)
X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

# Scaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Train: {len(X_train)} | Test: {len(X_test)}")
print(f"Features: {len(features)}")

Train: 372 | Test: 94
Features: 40


## 4. Entrenar Modelos con MLflow Tracking

In [8]:
# Definir modelos con sus hiperparámetros
modelos = {
    'Linear Regression': {
        'model': LinearRegression(),
        'params': {}
    },
    'Ridge': {
        'model': Ridge(alpha=1.0),
        'params': {'alpha': 1.0}
    },
    'Lasso': {
        'model': Lasso(alpha=0.01),
        'params': {'alpha': 0.01}
    },
    'SVR': {
        'model': SVR(kernel='rbf', C=10, epsilon=0.01),
        'params': {'kernel': 'rbf', 'C': 10, 'epsilon': 0.01}
    },
    'Random Forest': {
        'model': RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42),
        'params': {'n_estimators': 100, 'max_depth': 10}
    },
    'Gradient Boosting': {
        'model': GradientBoostingRegressor(n_estimators=100, max_depth=5, random_state=42),
        'params': {'n_estimators': 100, 'max_depth': 5}
    }
}

print(f"Modelos a entrenar: {len(modelos)}")

Modelos a entrenar: 6


In [9]:
# Entrenar cada modelo y registrar en MLflow
resultados = []

print("Entrenando modelos con MLflow tracking...\n")
print(f"{'Modelo':<25} {'RMSE':>10} {'MAE':>10} {'R²':>10}")
print("-" * 60)

for nombre, config in modelos.items():
    # Iniciar run de MLflow
    with mlflow.start_run(run_name=nombre):
        modelo = config['model']
        params = config['params']
        
        # Loggear parámetros
        mlflow.log_param("model_type", nombre)
        mlflow.log_param("n_features", len(features))
        mlflow.log_param("train_size", len(X_train))
        mlflow.log_param("test_size", len(X_test))
        for param_name, param_value in params.items():
            mlflow.log_param(param_name, param_value)
        
        # Entrenar
        if nombre == 'SVR':
            modelo.fit(X_train_scaled, y_train)
            y_pred = modelo.predict(X_test_scaled)
        else:
            modelo.fit(X_train, y_train)
            y_pred = modelo.predict(X_test)
        
        # Calcular métricas
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Loggear métricas
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("mae", mae)
        mlflow.log_metric("r2", r2)
        
        # Loggear modelo
        mlflow.sklearn.log_model(modelo, "model")
        
        # Guardar resultados
        resultados.append({
            'modelo': nombre,
            'RMSE': rmse,
            'MAE': mae,
            'R2': r2
        })
        
        print(f"{nombre:<25} {rmse:>10.4f} {mae:>10.4f} {r2:>10.4f}")

print("\n✓ Todos los modelos registrados en MLflow")

Entrenando modelos con MLflow tracking...

Modelo                          RMSE        MAE         R²
------------------------------------------------------------


2026/02/20 20:26:16 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/02/20 20:26:16 WARNING mlflow.sklearn: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization. The recommended safe alternative is the 'skops' format. For more information, see: https://scikit-learn.org/stable/model_persistence.html
2026/02/20 20:26:21 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/02/20 20:26:21 WARNING mlflow.sklearn: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization. The recommended safe alternative is the 'skops' format. For more information, see: https://scikit-learn.org/stable/model_p

Linear Regression             0.4633     0.2807    -1.6224


/Users/javiermondragon/Documents/data_projects/inflation-predictor/venv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.202e+00, tolerance: 2.455e-03
  model = cd_fast.enet_coordinate_descent(
2026/02/20 20:26:24 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/02/20 20:26:24 WARNING mlflow.sklearn: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization. The recommended safe alternative is the 'skops' format. For more information, see: https://scikit-learn.org/stable/model_persistence.html


Ridge                         0.4964     0.2941    -2.0095


...

Lasso                         0.3632     0.2416    -0.6117
SVR                           0.4046     0.3026    -1.0000


2026/02/20 20:26:31 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/02/20 20:26:31 WARNING mlflow.sklearn: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization. The recommended safe alternative is the 'skops' format. For more information, see: https://scikit-learn.org/stable/model_persistence.html


Random Forest                 0.2087     0.1533     0.4678


2026/02/20 20:26:36 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/02/20 20:26:36 WARNING mlflow.sklearn: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization. The recommended safe alternative is the 'skops' format. For more information, see: https://scikit-learn.org/stable/model_persistence.html


Gradient Boosting             0.2627     0.1783     0.1568

✓ Todos los modelos registrados en MLflow


## 5. Comparar Resultados

In [10]:
# Tabla de resultados
df_resultados = pd.DataFrame(resultados).sort_values('RMSE')
print("RANKING DE MODELOS")
print("=" * 60)
df_resultados

RANKING DE MODELOS


,modelo,RMSE,MAE,R2
4,Random Forest,0.208727,0.153278,0.467815
5,Gradient Boosting,0.262733,0.178297,0.156792
2,Lasso,0.363239,0.241568,-0.611725
3,SVR,0.404634,0.302614,-0.999999
0,Linear Regression,0.463337,0.280677,-1.622406
1,Ridge,0.496356,0.294139,-2.009488


In [11]:
# Mejor modelo
mejor = df_resultados.iloc[0]
print(f"\nMejor modelo: {mejor['modelo']}")
print(f"  RMSE: {mejor['RMSE']:.4f}")
print(f"  MAE: {mejor['MAE']:.4f}")
print(f"  R²: {mejor['R2']:.4f}")


Mejor modelo: Random Forest
  RMSE: 0.2087
  MAE: 0.1533
  R²: 0.4678


## 6. Ver en MLflow UI

Para ver los experimentos en la interfaz web de MLflow, corre esto en la terminal:

```bash
mlflow ui
```

Luego abre en el navegador: http://localhost:5000

In [ ]:
print("Para ver la UI de MLflow:")
print("1. Abre otra terminal")
print("2. Navega al proyecto: cd ~/Documents/data_projects/inflation-predictor")
print("3. Activa el entorno: source venv/bin/activate")
print("4. Corre: mlflow ui")
print("5. Abre: http://localhost:5000")

## 7. Registrar Mejor Modelo en Model Registry

In [ ]:
# Buscar el run del mejor modelo
experiment = mlflow.get_experiment_by_name(experiment_name)
runs = mlflow.search_runs(experiment_ids=[experiment.experiment_id])

# Filtrar por el mejor modelo
mejor_run = runs[runs['tags.mlflow.runName'] == mejor['modelo']].iloc[0]
mejor_run_id = mejor_run['run_id']

print(f"Mejor run ID: {mejor_run_id}")

In [ ]:
# Registrar modelo
model_name = "inflation-predictor"
model_uri = f"runs:/{mejor_run_id}/model"

# Registrar en Model Registry
result = mlflow.register_model(model_uri, model_name)

print(f"\n✓ Modelo registrado: {model_name}")
print(f"  Versión: {result.version}")
print(f"  Run ID: {mejor_run_id}")

## 8. Cargar Modelo desde Registry

In [ ]:
# Cargar modelo registrado
modelo_cargado = mlflow.sklearn.load_model(f"models:/{model_name}/1")

# Hacer predicción de prueba
y_pred_test = modelo_cargado.predict(X_test)

print(f"Modelo cargado desde registry")
print(f"Predicción de prueba (primeros 5):")
for i in range(5):
    print(f"  Real: {y_test.iloc[i]:.3f}% | Predicho: {y_pred_test[i]:.3f}%")

## 9. Resumen

In [ ]:
print("="*60)
print("RESUMEN DE MLFLOW")
print("="*60)

print(f"\n1. EXPERIMENTO: {experiment_name}")
print(f"   - Runs totales: {len(runs)}")
print(f"   - Tracking URI: {mlflow.get_tracking_uri()}")

print(f"\n2. MEJOR MODELO: {mejor['modelo']}")
print(f"   - RMSE: {mejor['RMSE']:.4f}")
print(f"   - MAE: {mejor['MAE']:.4f}")
print(f"   - R²: {mejor['R2']:.4f}")

print(f"\n3. MODEL REGISTRY:")
print(f"   - Nombre: {model_name}")
print(f"   - Versión: 1")

print(f"\n4. PARA VER LA UI:")
print(f"   mlflow ui")
print(f"   http://localhost:5000")

## Próximos Pasos

1. **Hyperparameter tuning:** Probar diferentes parámetros y trackear en MLflow
2. **Comparar en UI:** Ver gráficos de métricas en http://localhost:5000
3. **Promover modelo:** Mover de Staging a Production en el registry
4. **AWS:** Desplegar modelo registrado